In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="2"

import torch
from torch import nn
from torch.autograd import Variable

from data_loader import DataLoader
from model import UniSkip
from config import *
from datetime import datetime, timedelta\



# CUDA_VISIBLE_DEVICES=2torch.cuda.set_device(0)

USE_CUDA = torch.cuda.is_available()

w = torch.FloatTensor(2,3).cuda()

print(w)

tensor([[2.4231e+11, 4.5846e-41, 1.4552e-37],
        [0.0000e+00, 4.4842e-44, 0.0000e+00]], device='cuda:0')


In [2]:
d = DataLoader("./data/dummy_corpus.txt")

Loading text file at ./data/dummy_corpus.txt
Making dictionary for these words
Using cached dictionary at ./data/dummy_corpus.txt.pkl
Making reverse dictionary


In [3]:
mod = UniSkip()
# .cuda()
if USE_CUDA:
    mod.cuda() # 'cuda'

In [4]:
lr = 3e-4
optimizer = torch.optim.Adam(params=mod.parameters(), lr=lr)

In [5]:
loss_trail = []
last_best_loss = None
current_time = datetime.utcnow()

def debug(i, loss, prev, nex, prev_pred, next_pred):
    global loss_trail
    global last_best_loss
    global current_time

    this_loss = loss.data[0]
    loss_trail.append(this_loss)
    loss_trail = loss_trail[-20:]
    new_current_time = datetime.utcnow()
    time_elapsed = str(new_current_time - current_time)
    current_time = new_current_time
    print("Iteration {}: time = {} last_best_loss = {}, this_loss = {}".format(
              i, time_elapsed, last_best_loss, this_loss))
    
    print("prev = {}\nnext = {}\npred_prev = {}\npred_next = {}".format(
        d.convert_indices_to_sentences(prev),
        d.convert_indices_to_sentences(nex),
        d.convert_indices_to_sentences(prev_pred),
        d.convert_indices_to_sentences(next_pred),
    ))
    
    try:
        trail_loss = sum(loss_trail)/len(loss_trail)
        if last_best_loss is None or last_best_loss > trail_loss:
            print("Loss improved from {} to {}".format(last_best_loss, trail_loss))
            
            save_loc = "./saved_models/skip-best".format(lr, VOCAB_SIZE)
            print("saving model at {}".format(save_loc))
            torch.save(mod.state_dict(), save_loc)
            
            last_best_loss = trail_loss
    except Exception as e:
       print("Couldn't save model because {}".format(e))

In [ ]:
print("Starting training...")

# a million iterations
for i in range(0, 1000000):
    sentences, lengths = d.fetch_batch(32 * 8)

    loss, prev, nex, prev_pred, next_pred  = mod(sentences, lengths)
    

    if i % 10 == 0:
        debug(i, loss, prev, nex, prev_pred, next_pred)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    

Starting training...


/home/sumeetku/.local/lib/python3.5/site-packages/torch/nn/functional.py:995: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  # Remove the CWD from sys.path while we load stuff.
/home/sumeetku/deep_learning/skip-thoughts_sanyam5/data_loader.py:62: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  idx = idx.data[0]


Iteration 0: time = 0:00:01.379632 last_best_loss = None, this_loss = 19.813678741455078
prev = the zoo was good . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = the zoo was bad . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
pred_next = NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA NA
Loss improved from None to 19.813678741455078
saving model at ./saved_models/skip-best
Iteration 10: time = 0:00:03.665191 last_best_loss = 19.813678741455078, this_loss = 18.26944351196289
prev = zoo had a cat . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = the zoo was good . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = . . .

Iteration 140: time = 0:00:03.659973 last_best_loss = 16.722450256347656, this_loss = 16.187274932861328
prev = the zoo was good . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = the zoo was bad . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = the zoo was good . . . . . . . . . . . . . . . . . . . . . . . . . .
pred_next = the zoo was bad . . . . . . . . . . . . . . . . . . . . . . . . . .
Loss improved from 16.722450256347656 to 16.6867733001709
saving model at ./saved_models/skip-best
Iteration 150: time = 0:00:03.654063 last_best_loss = 16.6867733001709, this_loss = 16.18575668334961
prev = the zoo was good . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = the zoo was bad . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = the zoo was good . 

Iteration 280: time = 0:00:03.685557 last_best_loss = 16.189756393432617, this_loss = 16.18083953857422
prev = today i went to the zoo . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = zoo had a dog . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = today i went to the zoo . . . . . . . . . . . . . . . . . . . . . . . .
pred_next = zoo had a dog . . . . . . . . . . . . . . . . . . . . . . . . . .
Loss improved from 16.189756393432617 to 16.18772315979004
saving model at ./saved_models/skip-best
Iteration 290: time = 0:00:03.695017 last_best_loss = 16.18772315979004, this_loss = 16.176746368408203
prev = hi my name is sanyam . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = today i went to the zoo . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = hi my name is sanya

Iteration 420: time = 0:00:03.683454 last_best_loss = 16.180028915405273, this_loss = 16.181560516357422
prev = zoo had a dog . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = zoo had a cat . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = zoo had a dog . . . . . . . . . . . . . . . . . . . . . . . . . .
pred_next = zoo had a cat . . . . . . . . . . . . . . . . . . . . . . . . . .
Loss improved from 16.180028915405273 to 16.179838180541992
saving model at ./saved_models/skip-best
Iteration 430: time = 0:00:03.680031 last_best_loss = 16.179838180541992, this_loss = 16.181486129760742
prev = zoo had a dog . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = zoo had a cat . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = zoo had a dog . . . . . . . .

Iteration 560: time = 0:00:03.685582 last_best_loss = 16.177038192749023, this_loss = 16.17304229736328
prev = the zoo was bad . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = hi my name is sanyam . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = the zoo was bad . . . . . . . . . . . . . . . . . . . . . . . . . .
pred_next = hi my name is sanyam . . . . . . . . . . . . . . . . . . . . . . . . .
Loss improved from 16.177038192749023 to 16.176910400390625
saving model at ./saved_models/skip-best
Iteration 570: time = 0:00:03.682273 last_best_loss = 16.176910400390625, this_loss = 16.178180694580078
prev = today i went to the zoo . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = zoo had a dog . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = today i went to 

Iteration 710: time = 0:00:03.553966 last_best_loss = 16.17610740661621, this_loss = 16.172630310058594
prev = the zoo was bad . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = hi my name is sanyam . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = the zoo was bad . . . . . . . . . . . . . . . . . . . . . . . . . .
pred_next = hi my name is sanyam . . . . . . . . . . . . . . . . . . . . . . . . .
Iteration 720: time = 0:00:03.552322 last_best_loss = 16.17610740661621, this_loss = 16.172609329223633
prev = the zoo was bad . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = hi my name is sanyam . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = the zoo was bad . . . . . . . . . . . . . . . . . . . . . . . . . .
pred_next = hi my name is sanyam . . . . . . . 

Iteration 870: time = 0:00:03.554132 last_best_loss = 16.175615310668945, this_loss = 16.173681259155273
prev = hi my name is sanyam . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = today i went to the zoo . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = hi my name is sanyam . . . . . . . . . . . . . . . . . . . . . . . . .
pred_next = today i went to the zoo . . . . . . . . . . . . . . . . . . . . . . . .
Iteration 880: time = 0:00:03.556221 last_best_loss = 16.175615310668945, this_loss = 16.173667907714844
prev = hi my name is sanyam . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
next = today i went to the zoo . EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS EOS
pred_prev = hi my name is sanyam . . . . . . . . . . . . . . . . . . . . . . . . .
pred_next = today i went to the zoo . 